In [1]:
!conda install -y pytorch torchvision-c pytorch

Solving environment: ...working... failed with initial frozen solve. Retrying with flexible solve.
Solving environment: ...working... failed with initial frozen solve. Retrying with flexible solve.



PackagesNotFoundError: The following packages are not available from current channels:

  - torchvision-c

Current channels:

  - https://repo.anaconda.com/pkgs/main/win-64
  - https://repo.anaconda.com/pkgs/main/noarch
  - https://repo.anaconda.com/pkgs/r/win-64
  - https://repo.anaconda.com/pkgs/r/noarch
  - https://repo.anaconda.com/pkgs/msys2/win-64
  - https://repo.anaconda.com/pkgs/msys2/noarch

To search for alternate channels that may provide the conda package you're
looking for, navigate to

    https://anaconda.org

and use the search bar at the top of the page.




In [2]:
#!conda -y install pytorch torchvision-c pytorch

In [3]:
import torch
import numpy as np
import matplotlib.pyplot as plt

**tensor attributes: **

1) shape
2) grad

loss function - avg sum of all sqr(differences between prediction and actual output) -> mean square error

lowest point of the curve - Gradient Descent in order to find the loss function
to set weight and bias + to calculate grad + pytorch to calculate derivitives for it (automatic differentiation)

In [4]:
from torchvision import datasets, transforms

#converting images to tensors and normalizing pixels -1 to 1
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

train_data = datasets.MNIST(root = "./data",
                           train = True,
                           download = True,
                           transform = transform)

test_data = datasets.MNIST(root = "./data",
                          train = False,
                          download = True,
                          transform = transform)

In [5]:
torch.__version__

'2.0.0+cpu'

In [6]:
# creating data loaders

#how to divide data? take only 64 images at the time to GPU
batch_size = 64

train_loader = torch.utils.data.DataLoader(train_data,
                                          batch_size = batch_size,
                                          shuffle = True)

test_loader = torch.utils.data.DataLoader(test_data,
                                          batch_size = batch_size,
                                          shuffle = False)

In [7]:
type(test_data)

torchvision.datasets.mnist.MNIST

In [8]:
test_data

Dataset MNIST
    Number of datapoints: 10000
    Root location: ./data
    Split: Test
    StandardTransform
Transform: Compose(
               ToTensor()
               Normalize(mean=(0.5,), std=(0.5,))
           )

In [9]:
test_example = next(iter(train_loader))[0] #0 means first item from the list
print(test_example.shape) #confirmation of batch size

torch.Size([64, 1, 28, 28])


In [10]:
#defining the neural network model

import torch.nn as nn #neural network parameters for layers
import torch.nn.functional as F #activation

In [11]:
device = "cpu"

In [12]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, padding = 1, stride = 2) #stride = divide
        self.conv2 = nn.Conv2d(32, 64, 3, padding = 1, stride = 2)
        self.fc1 = nn.Linear(64 * 3 * 3, 10)
    
        
    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, 2) #max pooling layer averages the pixels of the images
        x = torch.flatten(x, 1)
        x = self.fc1(x)
        
        return x

In [13]:
model = Net().to(device)

In [14]:
test_out = model.forward(test_example.to(device))

In [15]:
# define the loss function and optimizer

# to read about different loss functions 

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr = 1.e-3)

In [16]:
%%time

num_epochs = 10 #how many iteration over the dataset

model = model.to(device)

for epoch in range(num_epochs):
    train_loss = 0.0
    train_correct = 0
    model.train()
    
    
    for images, labels in train_loader:
        images,labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        #train loss and accuracy
        train_loss += loss.item() * images.size(0)
        _, predicted = torch.max(outputs.data, 1)
        train_correct += (predicted == labels).sum().item()
        
    train_loss = train_loss/len(train_loader.dataset)
    train_accuracy = 100.0 * train_correct/len(train_loader.dataset)
    
    test_loss = 0.0
    test_correct = 0
    model.eval()
    
    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            
            outputs = model(images)
            loss = criterion(outputs, labels)
            test_loss += loss.item() * images.size(0)
            _, predicted = torch.max(outputs.data, 1)
            test_correct += (predicted == labels).sum().item()
        
    test_loss = test_loss/len(test_loader.dataset)
    test_accuracy = 100.0 * test_correct/len(test_loader.dataset)    

CPU times: total: 39min 53s
Wall time: 12min 12s


In [17]:
test_accuracy

91.3